In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

os.chdir('..')

# WritingProject

## 长文内容结构

In [2]:
from langchain_chinese import TreeContent, WritingTask
root = TreeContent()

t1 = root.add_item(TreeContent(title="H1", type="outline"))
t2 = root.add_item(TreeContent(title="H2",  type="outline"))

In [3]:
t2

'20240506.002714.00003.318'

In [4]:
h1 = root.get_item_by_id(t1)
h2_id = h1.add_item(TreeContent(text="my headline 1", title="Title1", type="outline"))
root.get_item_by_id(h2_id)

TreeContent(id='20240506.002714.00004.814', type='outline', is_completed=False, words_advice=None, title='Title1', howto=None, summarise=None, text='my headline 1', children=[], path=None)

In [5]:
root.all_todos()

[{'id': '20240506.002714.00001.333', 'type': 'outline', 'words_advice': None},
 {'id': '20240506.002714.00002.189', 'type': 'outline', 'words_advice': None},
 {'id': '20240506.002714.00004.814', 'type': 'outline', 'words_advice': None},
 {'id': '20240506.002714.00003.318', 'type': 'outline', 'words_advice': None}]

In [6]:
root.print_lines([1,6])

1.6.1 H1


1.6.1.1 Title1
my headline 1

1.6.2 H2




In [7]:
print(root.get_outlines([]))

1 H1 

1.1 Title1 

2 H2 



## 长文写作智能体

In [7]:
# 使用默认的智谱AI推理
wp = WritingTask(task_mode="askme")
wp.run()

task_mode: askme



👤:  帮我写1000字的农场童话，多使用适合儿童的语言对话，突出小动物们的幽默、可爱和调皮


-------------------- command: chat --------------------
-------------------- AI for 20240506.002825.00006.304 --------------------
resp: {'类型': 'outline', '标题名称': '农场小英雄', '扩写指南': '故事发生在一个充满欢乐的农场，小动物们各有特色，充满幽默、可爱和调皮。主要角色包括勇敢的小猪皮皮、聪明的猫咪妙妙、调皮的兔子跳跳等。', '总字数要求': 1000, '大纲数量': 4, '大纲列表': [{'标题名称': '第一章：农场的新早晨', '总字数要求': 250, '扩写指南': '描述农场的美景，小动物们起床的场景，突出它们的可爱和调皮。'}, {'标题名称': '第二章：小猪皮皮的冒险', '总字数要求': 250, '扩写指南': '讲述小猪皮皮的冒险经历，展现它的勇敢和智慧。'}, {'标题名称': '第三章：猫咪妙妙的计谋', '总字数要求': 250, '扩写指南': '描绘猫咪妙妙如何用计谋帮助小猪皮皮和其他小动物解决问题，突出它的聪明。'}, {'标题名称': '第四章：农场小英雄的胜利', '总字数要求': 250, '扩写指南': '描述小动物们团结一致，战胜困难，成为农场小英雄的过程。'}]}
-------------------- Todos Left --------------------
20240506.002825.00006.304 | words: None



👤:  给小动物们起一些名字吧


-------------------- command: chat --------------------
-------------------- AI for 20240506.002825.00006.304 --------------------
resp: {'类型': 'paragraph', '标题名称': '农场小动物们的名字', '总字数要求': 100, '详细内容': '农场里住着一群可爱的小动物，它们都有自己的名字。勇敢的小猪叫皮皮，聪明的猫咪叫妙妙，调皮的兔子叫跳跳，还有温柔的绵羊叫绵绵，勤劳的小马叫奔奔，乐观的小鸡叫欢欢，机灵的松鼠叫闹闹。', '内容摘要': '小动物们的名字：皮皮（小猪）、妙妙（猫咪）、跳跳（兔子）、绵绵（绵羊）、奔奔（小马）、欢欢（小鸡）、闹闹（松鼠）。'}
-------------------- Todos Left --------------------
20240506.002825.00006.304 | words: None


...

-------------------- command: ok --------------------
-------------------- Paragraph Done for 20240506.003216.00009.765 --------------------
-------------------- Next TODO for  20240506.003216.00010.836 --------------------
👤[auto]:  请帮我扩写《第四章：农场小英雄的胜利》, 字数约为250字，扩写依据为：描述小动物们团结一致，战胜困难，成为农场小英雄的过程。
-------------------- AI for 20240506.003216.00010.836 --------------------
resp: {'类型': 'paragraph', '标题名称': '第四章：农场小英雄的胜利', '总字数要求': 250, '详细内容': '在农场的某个角落，小鸡、小兔子和小猪皮皮面临着一个大难题——如何找回被风吹走的小鸡的帽子。这个困难让它们意识到，只有团结一致，才能战胜眼前的困境。在猫咪妙妙的带领下，它们分工合作，开始了一场拯救帽子的冒险。小猪皮皮的勇敢、小兔子的机灵、小鸡的坚持，以及妙妙的智慧，使它们克服了重重难关。最终，在农场的小山丘上，它们一起把帽子从小树梢上取下，赢得了农场其他动物的掌声。这一刻，它们不再是普通的小动物，而是农场小英雄，它们的友谊和合作成为了农场的传奇。', '内容摘要': {'地点': '农场', '实体': ['小鸡', '小兔子', '小猪皮皮', '猫咪妙妙'], '特点': '团结、勇敢、智慧、友谊', '场景': '合作冒险、战胜困难、赢得尊敬'}}
-------------------- Todos Left --------------------
20240506.003216.00010.836 | words: 250
20240506.003216.00011.158 | words: 200



👤:  ok


-------------------- command: ok --------------------
-------------------- Paragraph Done for 20240506.003216.00010.836 --------------------
-------------------- Next TODO for  20240506.003216.00011.158 --------------------
👤[auto]:  请帮我扩写《第五章：薛小乐的加入》, 字数约为200字，扩写依据为：介绍薛小乐这个角色，她的性格特点以及她与小动物们的友谊。
-------------------- AI for 20240506.003216.00011.158 --------------------
resp: {'类型': 'paragraph', '标题名称': '第五章：薛小乐的加入', '总字数要求': 200, '详细内容': '薛小乐，一个活泼开朗的小女孩，走进了农场这片充满生机的土地。她的笑声像清晨的阳光，洒满每一个角落。薛小乐热爱自然，性格中充满了温暖与耐心，她能与农场里的小动物们用心灵沟通。她最喜欢的事情就是坐在草地上，小鸡、小兔子、小猪围绕在她身边，它们的友谊无声而纯粹。在这些小动物眼中，薛小乐不仅是朋友，更是值得信赖的伙伴。', '内容摘要': {'地点': '农场', '实体': ['薛小乐', '小鸡', '小兔子', '小猪'], '特点': '活泼开朗、温暖耐心、与动物的友谊', '场景': '农场日常、互动交流、友谊建立'}}
-------------------- Todos Left --------------------
20240506.003216.00011.158 | words: 200



👤:  ok


-------------------- command: ok --------------------
-------------------- Paragraph Done for 20240506.003216.00011.158 --------------------
-------------------- Task Complete! --------------------


In [8]:
wp.print_lines()

1 第一章：农场的新早晨
晨曦徐徐拉开了帷幕，农场在一片金色的光辉中苏醒。空气中弥漫着清新的泥土香，露珠在翠绿的草叶上闪烁。小动物们开始陆续起床，毛茸茸的小鸡们叽叽喳喳地从温暖的窝里探出头来，小兔子们则耳朵摇摇地跳跃在青青的草地上。特别是那群小猪，他们哼哼哈哈地打着哈欠，胖乎乎的身体扭来扭去，显得格外可爱和调皮。他们相互追逐，打滚，完全不在意溅起的泥巴，给新一天的农场带来了生机与活力。

2 第二章：小猪皮皮的冒险
在农场的一角，小猪皮皮瞪大了它那双好奇的眼睛，准备开始它一天的冒险。它决定探索农场的深处，一个它从未涉足的地方。沿着曲折的小径，皮皮遇到了一个难题——一条湍急的小溪拦住了它的去路。然而，勇敢且智慧的小猪并没有放弃，它巧妙地利用周围的树枝和藤蔓编织成一座简易的桥，成功抵达了对岸。冒险并未结束，皮皮在森林里遇到了迷失方向的小兔子，它用自己的智慧带领小兔子找到了回家的路。这一天的经历，充分展现了小猪皮皮的勇敢和智慧。

3 第三章：猫咪妙妙的计谋
在农场的角落里，猫咪妙妙正用它那双晶莹剔透的眼睛，机智地观察着周围的一切。小猪皮皮和其他小动物遇到了难题，妙妙却沉着冷静地想出了一个计谋。它先是用一些小零食引诱小鸡和小兔子帮忙，然后巧妙地利用它们各自的特点，分配了任务。在妙妙的指挥下，皮皮负责挖掘，小兔子负责找寻，而小鸡则负责警示。它们配合得天衣无缝，问题很快得到了解决。猫咪妙妙的计谋不仅解决了小猪皮皮的困境，也让其他小动物对它的聪明才智赞叹不已。

4 第四章：农场小英雄的胜利
在农场的某个角落，小鸡、小兔子和小猪皮皮面临着一个大难题——如何找回被风吹走的小鸡的帽子。这个困难让它们意识到，只有团结一致，才能战胜眼前的困境。在猫咪妙妙的带领下，它们分工合作，开始了一场拯救帽子的冒险。小猪皮皮的勇敢、小兔子的机灵、小鸡的坚持，以及妙妙的智慧，使它们克服了重重难关。最终，在农场的小山丘上，它们一起把帽子从小树梢上取下，赢得了农场其他动物的掌声。这一刻，它们不再是普通的小动物，而是农场小英雄，它们的友谊和合作成为了农场的传奇。

5 第五章：薛小乐的加入
薛小乐，一个活泼开朗的小女孩，走进了农场这片充满生机的土地。她的笑声像清晨的阳光，洒满每一个角落。薛小乐热爱自然，性格中充满了温暖与耐心，她能与农场里的小动物们用心灵沟通。她最喜欢的事情就是坐在草地上，小鸡、小兔子、小猪

In [12]:
from langchain_openai import ChatOpenAI

# 使用 OpenAI/ChatGPT 推理
wp = WritingTask(task_mode="auto")
wp.run(llm = ChatOpenAI(model_name="gpt-4-turbo"))

task_mode: auto



👤:  帮我写1000字的农场童话，多使用适合儿童的语言对话，突出小动物们的幽默、可爱和调皮


-------------------- command: chat --------------------
-------------------- AI for 20240505.235936.00014.694 --------------------
resp: {'类型': 'outline', '标题名称': '农场童话', '扩写指南': '这个农场童话故事将包含幽默、可爱和调皮的小动物，如小猪、小羊、鸭子等。故事发生在一个充满乐趣的农场，强调友谊和团结的主题，使用适合儿童的语言。', '总字数要求': 1000, '大纲数量': 5, '大纲列表': [{'标题名称': '开场与介绍', '总字数要求': 200, '扩写指南': '介绍农场的环境和动物们的日常生活，突出他们的可爱和幽默，如鸭子在水塘的嬉戏，小猪的泥浆浴。'}, {'标题名称': '小动物们的特别任务', '总字数要求': 200, '扩写指南': '描述一个由农场主委托的特别任务，动物们如何协作，展现他们的智慧和调皮。'}, {'标题名称': '问题与冲突', '总字数要求': 200, '扩写指南': '任务中出现的意外情况，如小羊误入附近的森林，小动物们如何解决问题。'}, {'标题名称': '解决方案和团队协作', '总字数要求': 200, '扩写指南': '小动物们团结一致，每个动物根据自己的特长贡献力量，共同找到解决问题的办法。'}, {'标题名称': '结局与反思', '总字数要求': 200, '扩写指南': '任务成功完成后的庆祝，动物们和农场主的互相感谢，强调团队协作和友谊的重要性。'}]}
-------------------- Todos Left --------------------
20240505.235936.00014.694 | words: None
-------------------- command: ok --------------------
-------------------- Outlines Done for 20240505.235936.00014.694 --------------------
-------------------- Next TODO for  20240506.000139.0001

In [11]:
wp.print_lines()

1 风雨之夜的到来
在一个暴风雨的夜晚，约翰和汤姆决心探索镇上那栋有名的老房子。这座房子因其神秘的过去和诡异的传说而闻名。据说，几十年前，一位富有的商人在此神秘失踪，自那以后，无数的探险者和好奇者试图揭开房子的秘密，却无一例外地被各种怪异的事件和不幸的命运所阻挠。约翰和汤姆对超自然现象充满了兴趣，他们相信，通过他们的智慧和勇气，可以揭示这座老房子隐藏的真相。

他们的征途并不顺利。天色逐渐暗沉，狂风大作，雷声隆隆，仿佛大自然也在警告他们不要前进。当他们接近房子时，几个当地人劝他们回头，告诉他们许多人因好奇心而遭受不幸。但约翰和汤姆的决心未被动摇，他们相信这是一次生命中难得的冒险，即使是最恶劣的天气和最棘手的警告也阻挡不了他们的脚步。

2 秘密的低语声


2.1 进入老房子
在风雨交加的夜晚，约翰和汤姆站在那座传说中的老房子前，怀着忐忑不安的心情，他们推开了那扇沉重的前门。门吱嘎作响，仿佛在抗议着这对未被邀请的客人。一进入，一股阴冷及潮湿的空气迎面扑来，使人不寒而栗。房内光线昏暗，只有外面闪电的光芒偶尔透进来，照亮了房间里陈旧的家具和满墙的蛛网。两人试图适应这光线，四处观望，内心的恐惧与好奇心在此刻交织在一起。

2.2 低语声的发现
当约翰和汤姆勇敢地深入老房子的阴暗走廊时，一阵低沉而不连续的低语声突然在空气中回荡。这声音似乎从无处而来，却又无处不在，使得两人不由自主地停下脚步，屏息凝听。他们的眼神交换了一个忐忑的眼色，相互点头后，悄无声息地沿着声音的大致方向前进。汤姆轻轻地把耳朵贴近发霉的墙壁，试图捕捉那若有若无的低语，而约翰则试图通过手电筒的微弱光线，寻找可能的声源。在这种未知与恐惧的驱使下，他们小心翼翼地一步步接近那神秘的低语声。

2.3 追踪与恐惧
随着每一步深入，房子的古老木地板在约翰和汤姆的脚下吱吱作响。隐约的低语声引导他们穿过长长的走廊，每当他们靠近，一阵风似乎从窗外掠过，使得厚重的窗帘无声无息地摇晃。这些动静让房间笼罩在一种不安的气氛中，他们的心跳加速，恐惧与好奇心在胸中交织，推动他们继续前行，尽管内心深处不断有声音告诫他们退后。

3 揭开真相的时刻
最终，约翰和汤姆在房子的最深处，发现了低语声的来源。他们不禁停下脚步，目瞪口呆地望着眼前的场景。在昏暗的灯光下，一张古老的书桌上摆放着一个古怪的箱子，发出微弱的低语声。随着他们的靠近，低语声渐渐

In [10]:
print(wp.root_content.get_outlines())

1 第一章：重名的阴影 
介绍主要人物和背景，发现小说中的名字与现实中的同学相同，开始发生第一个意外事件。
1.1 开篇介绍 
故事背景：静枫中学，主要人物：李晓风，赵瑞，林雨晨，王子豪。
1.2 日常变奏 
主要人物李晓风在图书馆发现一本小说《死亡名单》，里面的角色名字与现实中的同学一模一样，引发了一连串的不安与怀疑。
1.3 阴影初现 
第一个意外事件：王子豪在打扫卫生时，无缘无故从楼梯上摔下，头部受创。这一事件引发主要人物们的不安与怀疑。
1.4 人心惶惶 
意外事件后，主要人物和周围同学都感受到了巨大的心理压力和恐惧，整个学校的氛围变得紧张不安。
2 第二章：恐惧蔓延 
角色们开始感到不安，第二个和第三个重名的同学也遭遇不幸，大家陷入恐慌。
2.1 2.1 不祥的征兆 
第二个重名同学赵瑞频繁遭遇诡异事件，李晓风和林雨晨对此感到担忧和恐惧，担心他会成为下一个不幸者。
2.2 2.2 恐慌的蔓延 
第二个重名同学赵瑞遭遇不幸事件，引发学校内恐慌情绪蔓延，同学们纷纷避开与小说角色重名的人，学校氛围变得沉重。主要人物们意识到必须找到解决方法。
2.3 2.3 灾难的连续 
第三个重名同学林雨晨在类似情况下遭遇不幸，导致主要角色们的恐慌达到顶点。人物：林雨晨，李晓风，王子豪。背景：静枫中学。设定：小说《死亡名单》中的角色与现实中的同学重名，引发连续不幸事件。
2.4 2.4 陷入混乱 
静枫中学的恐慌氛围，主要人物调查诅咒但陷入混乱，其他同学避开他们，调查变得更加困难。
3 第三章：揭秘行动 
主要角色决定调查这一系列事件的背后原因，发现小说的存在与一个古老的诅咒有关。
3.1 3.1 调查启动 
主要人物李晓风、赵瑞、林雨晨在图书馆展开首次调查会议，制定调查计划，分工合作，开始揭秘行动。涉及实体：李晓风、赵瑞、林雨晨、静枫中学图书馆。设定：揭开《死亡名单》背后的谜团，分工合作进行调查。
3.2 3.2 线索搜集 
人物：李晓风、赵瑞、林雨晨。地点：静枫中学图书馆。情节：角色们搜集到小说《死亡名单》与古老诅咒的联系。背景设定：静枫中学图书馆藏有关于诅咒的线索，学校可能隐藏着揭开诅咒的秘密。
3.3 3.3 诅咒揭秘 
在本段中，主要角色们在图书馆的古老资料中发现了《死亡名单》小说与一个古老诅咒的联系。这个诅咒源自几百年前一位被背叛的术士，诅咒的名字会传递并给继承者带

In [88]:
wp = WritingProject(streaming=True)
wp.run()


👤:  帮我写一个500字小故事


{}
{'类型': ''}
{'类型': 'out'}
{'类型': 'outlines'}
{'类型': 'outlines', '大纲列表': []}
{'类型': 'outlines', '大纲列表': [{}]}
{'类型': 'outlines', '大纲列表': [{'标题名称': ''}]}
{'类型': 'outlines', '大纲列表': [{'标题名称': '开'}]}
{'类型': 'outlines', '大纲列表': [{'标题名称': '开篇'}]}
{'类型': 'outlines', '大纲列表': [{'标题名称': '开篇', '字数要求': 100}]}
{'类型': 'outlines', '大纲列表': [{'标题名称': '开篇', '字数要求': 100, '内容摘要': ''}]}
{'类型': 'outlines', '大纲列表': [{'标题名称': '开篇', '字数要求': 100, '内容摘要': '描述'}]}
{'类型': 'outlines', '大纲列表': [{'标题名称': '开篇', '字数要求': 100, '内容摘要': '描述故事'}]}
{'类型': 'outlines', '大纲列表': [{'标题名称': '开篇', '字数要求': 100, '内容摘要': '描述故事背景'}]}
{'类型': 'outlines', '大纲列表': [{'标题名称': '开篇', '字数要求': 100, '内容摘要': '描述故事背景和'}]}
{'类型': 'outlines', '大纲列表': [{'标题名称': '开篇', '字数要求': 100, '内容摘要': '描述故事背景和主人公'}]}
{'类型': 'outlines', '大纲列表': [{'标题名称': '开篇', '字数要求': 100, '内容摘要': '描述故事背景和主人公'}, {}]}
{'类型': 'outlines', '大纲列表': [{'标题名称': '开篇', '字数要求': 100, '内容摘要': '描述故事背景和主人公'}, {'标题名称': ''}]}
{'类型': 'outlines', '大纲列表': [{'标题名称': '开篇', '字数要求': 100, '内容摘要': '描述故事背景和


👤:  quit


-------------------- quit
